#### Imports

In [2]:
from scipy import spatial, sparse
from scipy.stats import chi2
from collections import Counter
from num2words import num2words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.externals import joblib 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import KFold
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import os
import imp
import gzip
import copy
import nltk
import pickle
import scipy
import string
import gensim
import operator
import datetime

import numpy as np
import pandas as pd
import LDA_ELJST as lda
import matplotlib.pyplot as plt

In [3]:
import utils as my_utils
from sentiment import SentimentAnalysis

### Read Data

In [ ]:
dataset_name = "stackoverflow_fasttext_with_stopwords"

In [ ]:
dataset_questions = pd.read_csv("nongit_resources/Questions.csv", encoding='ISO-8859-1')

In [ ]:
dataset = pd.read_pickle("resources/data_stackoverflow_5kanswers_pd")

In [ ]:
dataset.head(2)

In [ ]:
dataset.shape

In [ ]:
dataset = dataset.rename(columns={'cleaned_body': 8, 'clean': 9})

In [ ]:
Counter(dataset['sent_score_quant'])

In [ ]:
count_matrix_, tfidf_matrix_, vocabulary, words = my_utils.processReviews(dataset[9].values)

In [ ]:
count_matrix_.shape, tfidf_matrix_.shape

In [ ]:
barren = np.where(count_matrix_.sum(1)==0)[0]

In [ ]:
barren

In [ ]:
count_matrix, tfidf_matrix, texts, ratings, ids = [], [], [], [], []

texts_ =  dataset['body_text'].values.tolist()
ratings_ = dataset['sent_score_quant'].values.tolist()
ids_ = dataset['Id'].values.tolist()

for idx, (i, j, k, l, m) in enumerate(zip(count_matrix_, tfidf_matrix_, texts_, ratings_, ids_)):
    if idx not in barren:
        count_matrix.append(i.tolist())
        tfidf_matrix.append(j.tolist())
        texts.append(k)
        ratings.append(l)
        ids.append(m)
        
count_matrix = np.array(count_matrix)
tfidf_matrix = np.array(tfidf_matrix)
texts = np.array(texts)
ratings = np.array(ratings)
ids = np.array(ids)

### Making Edge_Dict

In [ ]:
pickle_in = open("resources/docs_edges_stackoverflow_glove_5kanswers.pickle","rb")
docs_edges = pickle.load(pickle_in)

In [ ]:
docs_edges = np.delete(docs_edges, barren).tolist()

In [ ]:
len(docs_edges)

In [ ]:
edge_dict = {}
for i in docs_edges:
    for j in i:
        try:
            edge_dict[j[0]] += [j[1]]
            edge_dict[j[1]] += [j[0]]
        except:
            edge_dict[j[0]] = [j[1]]
            edge_dict[j[1]] = [j[0]]

In [ ]:
for i in edge_dict.keys():
    edge_dict[i] = list(set(edge_dict[i]))

In [ ]:
len(edge_dict)

## Run Model

In [ ]:
maxiter = 20
lambda_param = 1.0
N_ITERATAIONS = 5
N_SENTIMENT = 5

In [ ]:
dataset_name

In [ ]:
folder_name = str(datetime.datetime.now()) + "_" + dataset_name
os.mkdir("dumps/"+folder_name)

In [ ]:
# topics_grid = [5, 10, 25, 30,  35, 45, 50]

In [ ]:
import multiprocessing

In [ ]:
# def multiprocessing_func(k):
#     sampler = lda.LdaSampler(n_sentiment = N_SENTIMENT, n_topics=k, lambda_param=lambda_param)
#     sampler.store_data(count_matrix, ratings, words, vocabulary, docs_edges, edge_dict, ids)

#     print("Running Sampler...", k)
#     for i in range(N_ITERATAIONS):
#         print(datetime.datetime.now().time(), " iteration:", i, "k", k)
#         sampler.run(count_matrix, ratings, edge_dict, maxiter=maxiter)
#         joblib.dump(sampler, "dumps/" + folder_name + "/sampler_" + dataset_name + "_n_topics_" + str(k) + "_maxiter_"
#                     + str(maxiter) + "_iter_" + str(i+1) +"_in_" + str(N_ITERATAIONS))

In [ ]:
# pool = multiprocessing.Pool()

In [ ]:
# pool.map(multiprocessing_func, topics_grid)

# Evaluations

In [ ]:
# folder_name = "2019-08-16 23:49:40.458236_stackoverflow"

In [ ]:
# dataset_names = ['sampler_stackoverflow_n_topics_5_maxiter_20_iter_5_in_5',
#                 'sampler_stackoverflow_n_topics_10_maxiter_20_iter_5_in_5',
#                 'sampler_stackoverflow_n_topics_25_maxiter_20_iter_5_in_5',
#                 'sampler_stackoverflow_n_topics_30_maxiter_20_iter_5_in_5',
#                 'sampler_stackoverflow_n_topics_35_maxiter_20_iter_5_in_5',
#                 'sampler_stackoverflow_n_topics_45_maxiter_20_iter_5_in_5',
#                 'sampler_stackoverflow_n_topics_50_maxiter_20_iter_5_in_5']

In [ ]:
# from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

# def geometric_mean(x):
#     x = [i for i in x if i!=0]
#     return scipy.stats.mstats.gmean(x)

In [ ]:
# for i, k in zip(dataset_names, topics_grid):
#     sampler = joblib.load("dumps/" + folder_name + "/" + i)
#     data = pd.DataFrame()

#     data['text'] = texts
#     data['sent'] = ratings

#     count_matrix = sampler.get_count_matrix()
#     binary_mat = (count_matrix > 0).astype(int)
#     word_freq = binary_mat.sum(axis=0)

#     df_matrix = np.matmul(binary_mat, np.diag(word_freq))
#     df_matrix = df_matrix * 1.0/df_matrix.shape[0]

#     document_topic_entropy = scipy.stats.entropy(sampler.theta().transpose())
#     data['document_topic_entropy'] = document_topic_entropy

#     word_len = count_matrix.sum(axis=1)
#     data['word_len'] = word_len

#     data['document_popularity'] = np.array([geometric_mean(x) for x in df_matrix.tolist()])

#     normalized_dts = sampler.pi() * sampler.theta()[:,:,np.newaxis]
#     normalized_dts /= normalized_dts.sum(axis=-1)[:,:,np.newaxis]

#     document_topic_sentiment_crossentropy = np.array([[scipy.stats.entropy(j) for j in i] for i in normalized_dts])
#     document_ts_entropy_min = document_topic_sentiment_crossentropy.min(axis=1)
#     document_ts_entropy_mean = document_topic_sentiment_crossentropy.mean(axis=1)
#     document_ts_entropy_var = np.sqrt(document_topic_sentiment_crossentropy.var(axis=1))

#     data['document_ts_entropy_mean'] = document_ts_entropy_mean
#     data['document_ts_entropy_min'] = document_ts_entropy_min
#     data['document_ts_entropy_var'] = document_ts_entropy_var

#     data['num_edges'] = [len(i) for i in sampler.docs_edges]
#     data['Id'] = sampler.answer_id

#     topic_distances = 1/cosine_similarity(sampler.theta().transpose())
#     rao_score = []
#     for i in range(sampler.theta().shape[0]):
#         temp = np.outer(sampler.theta()[i],sampler.theta()[i]) * topic_distances
#         rao_score.append(0.5*temp.sum()+0.5*temp.trace())
#     rao_score = np.array(rao_score)

#     data['rao'] = rao_score

#     merged = data.merge(dataset, on='Id', how='inner')

#     dataset_questions = dataset_questions.rename(columns={'Id': 'ParentId'})

#     merged_questions = merged.merge(dataset_questions, on='ParentId', how='inner')

#     merged_questions.drop(columns=['text', 'sent', 8, 'CreationDate_y', 'CreationDate_x', 'Score_y', 
#                                    'Body_x']).rename(columns={'answererID': 'answerID', 'OwnerUserId_x': 'OwnerUserId', 
#                                                               'Score_x': 'answer_score', 'body_text': 'answer', 9: 'answer_clean', 'OwnerUserId_y': 'QuestionID', 'Title': 'Questions_Title', 'Body_y': 'Question_body'}).reset_index().to_pickle("dumps/" + folder_name + "/eval_tests_stackoverflow_pd_topics_" + str(k), protocol=2)

# Appendix

In [ ]:
# !pip install numpy num2words nltk pandas Observations gensim
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# !pip install -U textblob
# # !python -m textblob.download_corpora
# import nltk
# nltk.download('sentiwordnet')

In [ ]:
# df_matrix = scipy.sparse.load_npz("resources/df_stackoverflow_5kanswers.npz")

In [ ]:
# sampler = joblib.load("dumps/" + folder_name + "/" + i)
# data = pd.DataFrame()

# data['text'] = text_folds[idx]
# data['sent'] = sent_folds[idx]

# test_set = sampler.test_sentiment.shape[0]
# count_matrix = sampler.get_count_matrix()
# binary_mat = (count_matrix > 0).astype(int)
# word_freq = binary_mat.sum(axis=0)

# df_matrix = np.matmul(binary_mat, np.diag(word_freq))
# df_matrix = df_matrix * 1.0/df_matrix.shape[0]

# document_topic_entropy = scipy.stats.entropy(sampler.theta().transpose())
# data['document_topic_entropy'] = document_topic_entropy

# word_len = count_matrix.sum(axis=1)
# data['word_len'] = word_len

# document_popularity = np.true_divide(df_matrix.sum(1),(df_matrix != 0).sum(1)) 
# #scipy.stats.mstats.gmean(idf_matrix,axis=1)
# data['document_popularity'] = document_popularity

# normalized_dts = sampler.pi() * sampler.theta()[:,:,np.newaxis]
# normalized_dts /= normalized_dts.sum(axis=-1)[:,:,np.newaxis]

# document_topic_sentiment_crossentropy = np.array([[scipy.stats.entropy(j) for j in i] for i in normalized_dts])
# document_ts_entropy_min = document_topic_sentiment_crossentropy.min(axis=1)
# document_ts_entropy_mean = document_topic_sentiment_crossentropy.mean(axis=1)
# document_ts_entropy_var = np.sqrt(document_topic_sentiment_crossentropy.var(axis=1))

# data['document_ts_entropy_mean'] = document_ts_entropy_mean
# data['document_ts_entropy_min'] = document_ts_entropy_min
# data['document_ts_entropy_var'] = document_ts_entropy_var

# data['num_edges'] = [len(i) for i in sampler.docs_edges]
# data['answererID'] = sampler.answer_id

In [ ]:
# merged = pd.concat(csv_dumps).merge(dataset, on='answererID', how='inner')

# merged.drop(columns=['text', 'sent', 8]).reset_index().to_pickle("dumps/" + folder_name + "/eval_tests_amazon_QA_Electronics_pd", protocol=2)

# merged.shape

# merged_questions = merged.merge(dataset_questions, on='ParentId', how='inner')

# merged_questions.keys()

# merged_questions

# merged_questions.drop(columns=['text', 'sent', 8, 'CreationDate_y', 'CreationDate_x', 'Score_y', 'Body_x']).rename(columns={'answererID': 'answerID', 'OwnerUserId_x': 'OwnerUserId', 'Score_x': 'answer_score', 'body_text': 'answer', 9: 'answer_clean', 'OwnerUserId_y': 'QuestionID', 'Title': 'Questions_Title', 'Body_y': 'Question_body'}).reset_index().to_pickle("dumps/" + folder_name + "/eval_tests_stackoverflow_answers_pd", protocol=2)

In [ ]:
# sents_val = (dataset['sent1'].values*1000).astype(int)
# freq = []
# val = []
# for i in p:
#     if i != 0 and i > -200 and i < 200:
#         freq.append(p[i])
#         val.append(i)

# plt.plot(val, freq)

# len(sents_val)/5

# sents_range = []
# for i in sents_val:
#     if i < -10:
#         sents_range.append(1)
#     elif i < 1:
#         sents_range.append(2)
#     elif i < 22:
#         sents_range.append(3)
#     elif i < 52:
#         sents_range.append(4)
#     else:
#         sents_range.append(5)

# Counter(sents_range)

# ratings = np.array(sents_range)

# dataset['sent_range'] = sents_range

In [ ]:
# from math import floor
# import nltk
# from nltk.corpus import sentiwordnet as swn
# from nltk.tag.perceptron import PerceptronTagger
# from sklearn.metrics import accuracy_score
# from sklearn.pipeline import Pipeline

# tagger = PerceptronTagger()

# def compute_sentiment(sentence):
#     taggedsentence = []
#     sent_score = []
#     taggedsentence.append(tagger.tag(sentence.split()))
#     wnl = nltk.WordNetLemmatizer()
#     for idx, words in enumerate(taggedsentence):
#         for idx2, t in enumerate(words):
#             newtag = ''
#             lemmatizedsent = wnl.lemmatize(t[0])
#             if t[1].startswith('NN'):
#                 newtag = 'n'
#             elif t[1].startswith('JJ'):
#                 newtag = 'a'
#             elif t[1].startswith('V'):
#                 newtag = 'v'
#             elif t[1].startswith('R'):
#                 newtag = 'r'
#             else:
#                 newtag = ''
#             if (newtag != ''):
#                 synsets = list(swn.senti_synsets(lemmatizedsent, newtag))
#                 score = 0.0
#                 if (len(synsets) > 0):
#                     for syn in synsets:
#                         score += syn.pos_score() - syn.neg_score()
#                     sent_score.append(score / len(synsets))
#         if (len(sent_score)==0 or len(sent_score)==1):
#             return (float(0.0))
#         else:
#             return (sum([word_score for word_score in sent_score]) / (len(sent_score)))

In [ ]:
# def loadGloveModel(gloveFile):
#     print("Loading Glove Model")
#     f = open(gloveFile,'r')
#     model = {}
#     for line in f:
#         splitLine = line.split()
#         word = splitLine[0]
#         embedding = np.array([float(val) for val in splitLine[1:]])
#         model[word] = embedding
#     print("Done.",len(model)," words loaded!")
#     return model

In [ ]:
# %%time
# embeddings_index = loadGloveModel("nongit_resources/glove.42B.300d.txt")

In [ ]:
# %%time
# embeddings_index = gensim.models.KeyedVectors.load_word2vec_format("nongit_resources/electronics.vec")

In [ ]:
# %%time
# edges_threshold = 0.8
# docs_edges, ignored, taken, count = [], [], [], 0
# for idx, doc in enumerate(dataset[8].values):
#     edges = []
#     print(idx)
#     for i in doc:
#         for j in doc:
#             if i != j:
#                 try:
#                     a = embeddings_index[i]
#                     b = embeddings_index[j]
#                     if get_cosine(a, b) > edges_threshold and (vocabulary[i], vocabulary[j]) not in edges and (vocabulary[j], vocabulary[i]) not in edges:
#                         edges.append((vocabulary[i], vocabulary[j]))
#                 except:
#                     try:
#                         embeddings_index[i]
#                         taken.append(i)
#                     except:
#                         ignored.append(i)
#                     try:
#                         embeddings_index[j]
#                     except:
#                         ignored.append(j)
#                         taken.append(j)
#                     pass
#     docs_edges.append(edges)

In [ ]:
# pickle_out = open("resources/docs_edges_" + dataset_name + "_5k_fasttext_trained.pickle","wb")
# pickle.dump(docs_edges, pickle_out)
# pickle_out.close()

In [ ]:
## Sentence wise
# dataset = dataset[['asin', 'helpful', 'overall', 'reviewText']]
# dataset['n_words'] = dataset['reviewText'].apply(lambda x: len(w_tokenizer.tokenize(x)))
# dataset['sentences'] = dataset['reviewText'].apply(lambda x: [i.strip() for i in x.split(".")])
# dataset['sentence_word_density'] = dataset['reviewText'].apply(lambda x: len(w_tokenizer.tokenize(x))/ len(x.split(".")))
# dataset.to_csv("reviews_Musical_Instruments_5.csv")

In [ ]:
# edge_dict, ignored, taken, count = {}, [], [], 0
# for idxi, i in enumerate(vocabulary.keys()):
#     print(idxi)
#     for idxj, j in enumerate(vocabulary.keys()):
#         if i != j:
#             try:
#                 a = embeddings_index[i]
#                 b = embeddings_index[j]
#                 if get_cosine(a, b) > edges_threshold:
#                     try:
#                         edge_dict[vocabulary[i]] += [vocabulary[j]]
#                         edge_dict[vocabulary[j]] += [vocabulary[i]]
#                     except:
#                         edge_dict[vocabulary[i]] = [vocabulary[j]]
#                         edge_dict[vocabulary[j]] = [vocabulary[i]]
#             except:
#                 try:
#                     embeddings_index[i]
#                     taken.append(i)
#                 except:
#                     ignored.append(i)
#                 try:
#                     embeddings_index[j]
#                 except:
#                     ignored.append(j)
#                     taken.append(j)
#                 pass

In [ ]:
# df = {}

# for idx, i in enumerate(dataset[8].values):
#     print(idx)
#     for j in i:
#         try:
#             df[j] += [idx]
#         except:
#             df[j] = [idx]

# for i in df.keys():
#     df[i] = len(list(set(df[i])))

# df_vector = []
# for i in dataset[8].values:
#     d = [0]*len(vocabulary.keys())
#     for j in i:
#         if j in vocabulary.keys():
#             d[vocabulary[j]] = df[j]
#     df_vector.append(d)

In [ ]:
# csr = sparse.csr_matrix(np.array(df_vector))
# scipy.sparse.save_npz('resources/df_amazon_baby_sparse_200questions.npz', csr)

In [ ]:
# np.array(scipy.sparse.load_npz('resources/df_amazon_baby_sparse_200questions.npz').todense())

In [ ]:
# dataset = parse("nongit_resources/reviews_Electronics_5.json.gz")
# dataset = pd.DataFrame(list(dataset))
# dataset = dataset.head(N_docs)
# dataset.to_pickle("resources/reviews_Electronics_5")